# Llama 3.1 Rag Agent with LlamaIndex

<a target="_blank" href="https://colab.research.google.com/github/ytang07/ai_agents_cookbooks/blob/main/llamaindex/llama31_8b_rag_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook will walk you through building a LlamaIndex ReactAgent using Llama 3.1 70b. We will be using [OctoAI](https://octo.ai) as our embeddings and llm provider.

## Install Dependencies

In [1]:
! pip install -qU llama-index llama-index-llms-openai llama-index-readers-file octoai llama-index-llms-octoai llama-index-embeddings-octoai llama-index-embeddings-openai llama-index-llms-openai-like

! pip freeze llama-index-core
! pip freeze embeddings-openai

!pip install load_dotenv
!pip install selenium
!pip install Service
!pip install Options
!pip install By

'grep' is not recognized as an internal or external command,
operable program or batch file.
'grep' is not recognized as an internal or external command,
operable program or batch file.


## Setup API Keys
To run the rest of the notebook you will need access to an OctoAI API key. You can sign up for an account [here](https://octoai.cloud/). If you need further guidance you can check OctoAI's [documentation page](https://octo.ai/docs/getting-started/how-to-create-octoai-access-token).

In [3]:
from os import environ
from getpass import getpass
environ["OCTOAI_API_KEY"] = getpass("Input your OCTOAI API key: ")
from dotenv import load_dotenv

load_dotenv()

OCTOAI_API_KEY = environ["OCTOAI_API_KEY"]

Input your OCTOAI API key: ··········


## Import libraries and setup LlamaIndex

In [4]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.octoai import OctoAIEmbedding
from llama_index.core import Settings as LlamaGlobalSettings
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai_like import OpenAILike

# Set the default model to use for embeddings
LlamaGlobalSettings.embed_model = OctoAIEmbedding()

# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAILike(
    model="meta-llama-3.1-70b-instruct",
    api_base="https://text.octoai.run/v1",
    api_key=environ["OCTOAI_API_KEY"],
    context_window=40000,
    is_function_calling_model=True,
    is_chat_model=True,
)


## Load Documents

In [5]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/disease"
    )
    disease_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/treatments"
    )
    treatment_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [6]:
#if not index_loaded:

import os

# Get the current directory
current_directory = os.getcwd()

# load data
disease_docs = SimpleDirectoryReader(
    input_files=["./Disease_symptom_and_patient_profile_dataset_flat.csv"]
).load_data()

treatments_docs = SimpleDirectoryReader(
    input_files=["./Disease_treatments_flat.csv"]
).load_data()

# build index
disease_index = VectorStoreIndex.from_documents(disease_docs, show_progress=True)
treatments_index = VectorStoreIndex.from_documents(treatments_docs, show_progress=True)

# persist index
disease_index.storage_context.persist(persist_dir="./storage/disease")
treatments_index.storage_context.persist(persist_dir="./storage/treatments")


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/14 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Now create the query engines.

In [7]:
disease_engine = disease_index.as_query_engine(similarity_top_k=3, llm=llm)
treatments_engine = treatments_index.as_query_engine(similarity_top_k=3, llm=llm)


We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=disease_engine,
        metadata=ToolMetadata(
            name="disease_csv",
            description=(
                "Provides symptom and disease data from many patients."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=treatments_engine,
        metadata=ToolMetadata(
            name="treatments_csv",
            description=(
                "Provides diseases and matching treatment recommendations."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [9]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [10]:
response = agent.chat("I have a fever, a cough, I'm female, and my age is less than 40. What is my most likely disease? Also, what do you recommend for treatment?")
print(str(response))

> Running step c2424223-6b3b-42aa-9a6d-31ba8194e548. Step input: I have a fever, a cough, I'm female, and my age is less than 40. What is my most likely disease? Also, what do you recommend for treatment?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: disease_csv
Action Input: {'input': "I have a fever, a cough, I'm female, and my age is less than 40. What is my most likely disease?"}
Observation: Based on the information provided, you have a fever, a cough, you're female, and your age is less than 40. Considering these symptoms, the most likely diseases that match your profile are Asthma, Influenza, and Bronchitis.
> Running step 580afe62-34e8-474a-9df3-b11636160082. Step input: None
Thought: I have the most likely diseases, but I need to find the most likely one. I will use the same tool to get more information.
Action: disease_csv
Action Input: {'input': "I have a fever, a cough, I'm female, and my age is less than

ValueError: Reached max iterations.

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Set up headless Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Path to your ChromeDriver
chrome_driver_path = './chromedriver/chromedriver.exe'

# Create a Chrome WebDriver instance with the above options
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Target URL
url = 'https://example.com'  # Replace with the desired URL

# Open the URL
driver.get(url)

# Find the first image on the page
try:
    first_image = driver.find_element(By.TAG_NAME, 'img')
    image_url = first_image.get_attribute('src')
    print(f'First image URL: {image_url}')
except Exception as e:
    print(f"Error occurred: {e}")

# Quit the driver
driver.quit()


WebDriverException: Message: 'chromedriver.exe' executable may have wrong permissions.


In [92]:
import requests
from PIL import Image
from io import BytesIO

def display_image_from_url(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful

        # Open the image from the content of the response
        img = Image.open(BytesIO(response.content))

        # Display the image
        img.show()

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the image: {e}")

# Example usage: Replace with a valid Google Images URL
image_url = 'https://wpassets.halowaypoint.com/wp-content/2022/02/Halo-Infinite-Chief-Weapon-scaled.jpg'
display_image_from_url(image_url)